In [19]:
#IMPORTAR PACKAGES y LIBRARYS

#Librerías útiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
from scipy.signal import periodogram
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn import preprocessing
from itertools import combinations
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm
from pmdarima.utils import diff_inv

#otros
import warnings
warnings.filterwarnings("ignore")

#Modelos de Pruebas Estadísticas
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tools.eval_measures import mse,rmse,rmspe,bias,meanabs
import statsmodels.tsa.stattools as ts

#Modelos predictivos estadísticos
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.api import VAR

#Configuraciones
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
#DICCIONARIOS, OBJETOS
dic_mes = {
        'Enero': 1,
        'Febrero': 2,
        'Marzo': 3,
        'Abril':4,
         'Mayo':5,
         'Junio':6,
         'Julio':7,   
         'Agosto':8,
         'Setiembre':9,
         'Octubre':10,
         'Noviembre':11,
         'Diciembre':12
        }


In [21]:
#PREPARACIÓN_DATA

df= pd.read_excel("C:/Users/Usuario/OneDrive - prediqtdata.com/Desktop/Renato/Python/script atenea/consolidado_tradicional.xlsx"
,sheet_name="Tradicional aceites")
df.replace(dic_mes, inplace=True)
df['Quarter'] = df['MES'].apply(lambda x: 'Quarter1' if x in [1, 2, 3] else 'Quarter2' if x in [4, 5, 6] else 'Quarter3' if x in [7, 8, 9] else 'Quarter4')
df['Month_in_Quarter'] = df['MES'].apply(lambda x: 1 if x % 3 == 1 else 2 if x % 3 == 2 else 3)
df["PERIODO"]= df["AÑO"].astype(str)+"-"+df["MES"].astype(str)

df['PERIODO'] = pd.to_datetime(df['PERIODO'])    
#df['PERIODO'] = df['PERIODO'].dt.to_period('M')
df = df.set_index('PERIODO')

cols=['PERIODO','AÑO','MES','Quarter','Month_in_Quarter','OFICINA DE VENTAS'
      ,'VENTA SELL IN', 'VENTA SELL OUT TOTAL'
      , 'SHARE VOLUMEN ALICORP', 'SHARE VOLUMEN MERCADO'
      ,'SHARE % ALICORP','SHARE % MERCADO']

cols_data =['VENTA SELL IN'
      , 'VENTA SELL OUT TOTAL'
      #, 'SHARE VOLUMEN ALICORP'
      #,'SHARE VOLUMEN MERCADO'
      #,'SHARE % ALICORP'
      #,'SHARE % MERCADO'
      ]

df=df[cols_data]


In [22]:

#Modelamiento VAR
df_var = df                      

pred_dict = {'date':[], 'VENTA_SELL_IN_pred':[], 'ci_lower':[],'ci_upper':[] }

nobs= 18

for i in tqdm(range(nobs)):
    
    training_data = df_var[:(-nobs+i)]
   
    # VAR model selects lag p based on AIC
    
    model = VAR(training_data)
    model_fit = model.fit(ic = 'aic')
    model_results = model_fit.forecast_interval( y=training_data.to_numpy() , steps=1)

    pred_date = training_data.index[-1] + relativedelta(months = 1)
    pred_dict['date'].append(pred_date)
    pred_dict['VENTA_SELL_IN_pred'].append(model_results[0][0][0])
    pred_dict['ci_lower'].append(model_results[1][0][0])
    pred_dict['ci_upper'].append(model_results[2][0][0])



# transform data
df_pred = pd.DataFrame(pred_dict)
df_pred = df_pred.set_index('date')


df_pred= pd.concat([df_var,df_pred],axis=1,join="outer")



n=df_pred.shape[0]

plt.plot(df_pred['VENTA SELL IN'], color = 'green')
plt.plot(df_pred['VENTA_SELL_IN_pred'], color = 'red')
plt.legend(['VENTA SELL IN', 'VENTA_SELL_IN_pred'])
plt.fill_between(df_pred.index,
                 df_pred['ci_lower'],
                df_pred['ci_upper'], color='lightblue', alpha=0.5)
plt.title('VAR, MSE: {:,}'.format(round(mean_squared_error(
    df_pred['VENTA SELL IN'][-nobs:], df_pred['VENTA_SELL_IN_pred'][-nobs:]), 2)))
plt.grid()
plt.show()
MAPE= mean_absolute_percentage_error(df_pred['VENTA SELL IN'][-nobs:], df_pred['VENTA_SELL_IN_pred'][-nobs:])
print("MAPE:", round(MAPE,3))


model = VAR(df_var)
model_fit = model.fit(ic = 'aic')
model_results = model_fit.forecast_interval( y=training_data.to_numpy() , steps=1)

prediccion= model_results[0][0][0]

print("prediccion:",prediccion)

  0%|          | 0/18 [00:00<?, ?it/s]

IndexError: index 0 is out of bounds for axis 0 with size 0